In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from video import *
import numpy as np
import ultralytics
import cv2 as cv
from utils import *
from tqdm import tqdm 
from ultralytics import RTDETR
from multiprocessing import Pool

In [ ]:
fundamental_matrices = {
    "A" : {
        "B": np.load("fundamental_matrices/AB.npy"),
        "C": np.load("fundamental_matrices/AC.npy"),
    },
    "B": {
        "A": np.load("fundamental_matrices/BA.npy"),
        "C": np.load("fundamental_matrices/BC.npy"),
    },
    "C": {
        "B": np.load("fundamental_matrices/CB.npy"),
        "A": np.load("fundamental_matrices/CA.npy"),
    }
}

In [ ]:
homography_matrices = {
    "A" : {
        "B": np.load("homography_matrices/AB.npy"),
        "C": np.load("homography_matrices/AC.npy"),
    },
    "B": {
        "A": np.load("homography_matrices/BA.npy"),
        "C": np.load("homography_matrices/BC.npy"),
    },
    "C": {
        "B": np.load("homography_matrices/CB.npy"),
        "A": np.load("homography_matrices/CA.npy"),
    }
}

In [ ]:
video = "05"

In [ ]:
if 'query' in globals():
    del query
if 'reference' in globals():
    del reference

a = load_video(f"train/task2/{video}_1.mp4")
b = load_video(f"train/task2/{video}_2.mp4")
print(a.num_frames())
print(b.num_frames())

In [ ]:
def read_box(path):
    with open(path, "r") as f:
        lines = f.readlines()
    _, box = lines

    box = box.strip()
    _index, x1, y1, x2, y2 = list(map(int , box.split(" ")))
    return (x1, y1, x2, y2)

In [ ]:
initial_bbox = read_box(f"train/task2/{video}_1.txt")
plot = a.get_frame(0).raw().copy()
cv.rectangle(plot, initial_bbox[:2], initial_bbox[2:], (0, 255, 0), 2)

plt.figure(figsize=(10, 10))
plt.title("The first bounding box")
plt.imshow(plot)

In [ ]:
a.do_tracking(visualize=True)
b.do_tracking(visualize=True)

In [ ]:
def find_best_intersection(objects, bounding_box, threshold=0.5):
    max_iou = 0
    best = None

    for object in objects:
        bb = object["bbox"] 

        iou = bb_intersection_over_union(bb, bounding_box)
        if iou > threshold and iou > max_iou:
            best = object
            max_iou = iou

    return best 

In [ ]:
def find_closest_object(objects, position, threshold=100):
    min_distance = np.inf
    best = None

    for object in objects:
        bb = object["bbox"]
        center= bounding_box_center(bb) 
        distance = np.linalg.norm(np.array(center) - np.array(position))

        if distance< threshold and distance < min_distance:
            best = object
            min_distance = distance
    return best

In [ ]:
kalman = cv.KalmanFilter(8, 4)
kalman.measurementMatrix = np.array(
            [[1, 0, 0, 0, 0, 0, 0, 0],
             [0, 1, 0, 0, 0, 0, 0, 0],
             [0, 0, 1, 0, 0, 0, 0, 0],
             [0, 0, 0, 1, 0, 0, 0, 0],
             ], np.float32)

kalman.transitionMatrix = np.array(
            [[1, 0, 0, 0, 1, 0, 0, 0],
             [0, 1, 0, 0, 0, 1, 0, 0],
             [0, 0, 1, 0, 0, 0, 1, 0],
             [0, 0, 0, 1, 0, 0, 0, 1],
             [0, 0, 0, 0, 1, 0, 0, 0],
             [0, 0, 0, 0, 0, 1, 0, 0],
             [0, 0, 0, 0, 0, 0, 1, 0],
             [0, 0, 0, 0, 0, 0, 0, 1]
            ], np.float32)

kalman.processNoiseCov = np.eye(8, dtype=np.float32) * 0.01

kalman.measurementNoiseCov = np.array(
            [[30, 0, 0, 0],
             [0, 30, 0, 0],
             [0, 0, 200, 0],
             [0, 0, 0, 200]], np.float32)

kalman.statePre = np.array([[initial_bbox[0]],
                            [initial_bbox[1]],
                            [initial_bbox[2] - initial_bbox[0]],
                            [initial_bbox[3] - initial_bbox[1]],
                            [0],
                            [0],
                            [0],
                            [0]], np.float32)

kalman.statePost = np.array([[initial_bbox[0]],
                            [initial_bbox[1]],
                            [initial_bbox[2] - initial_bbox[0]],
                            [initial_bbox[3] - initial_bbox[1]],
                            [0],
                            [0],
                            [0],
                            [0]], np.float32)

for frame_index in range(0, a.num_frames()):
    frame = a.get_frame(frame_index)
    plot = frame.raw().copy()
    objects = frame.objects()

    for object in objects:
        cv.rectangle(plot, object["bbox"][:2], object["bbox"][2:], (0, 255, 0), 2)

    prediction = kalman.predict()[:4].flatten().astype(np.int32)
    prediction = (prediction[0], prediction[1], prediction[0] + prediction[2], prediction[1] + prediction[3])

    # Proritize finding the best intersection with existing objects
    best_intersection = find_best_intersection(objects, prediction, threshold=0.3)

    found = None 
    if best_intersection is not None:
        found = best_intersection["bbox"]
    else:
        # If that fails, find the closest pbject.
        center = bounding_box_center(prediction)
        closest_object = find_closest_object(objects, center, threshold=50)
        if closest_object is not None:
            found = closest_object["bbox"]

    if found:
        kalman.correct(np.array([[found[0]], [found[1]], [found[2] - found[0]], [found[3] - found[1]]], np.float32))

    cv.rectangle(plot, prediction[:2], prediction[2:], (255, 0, 0), 2)
    cv.imshow("tracking", plot)
    cv.waitKey(100)

cv.destroyAllWindows()